## This notebook looks at the sensitivity of A04N simulations to:

1. Modified debye relationship
2. Basal air gap
3. Density and SSA range
4. Presence of a surface crust
5. Snow depth

In [ ]:
import numpy as np
#%matplotlib notebook
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import glob
from matplotlib.lines import Line2D
from datetime import datetime, timedelta
from scipy.stats import linregress
import scipy.io as sio
from copy import deepcopy
%load_ext autoreload

from smrt_aesop_setup import return_snowpack
from run_smrt_with_realistic_atmosphere import get_atmosphere, tb_results

In [ ]:
matfile = sio.loadmat('../Data/ground_obs/TVC2018_trenches.mat',struct_as_record=False, squeeze_me=True)
A04N = matfile['TVC'].A04N

In [ ]:
# Make snowpacks
print (A04N.layer_thickness.layer_thickness) # layer_thickness
print (A04N.ice_lens.ice_lens_height_top) # ice_loc

In [ ]:
# SMRT import statements
# SMRT imports
from smrt import make_snowpack, make_model, sensor_list, make_soil
from smrt.core.globalconstants import DENSITY_OF_ICE, FREEZING_POINT

In [ ]:
# Get atmosphere
atmosphere = get_atmosphere('C087', 'A04', single_frequency=True)

# Construct snowpack with ice lens
base_snowpack = return_snowpack(A04N, with_ice_lens=True,flight_temperature_correction='C087',
                                atmosphere=atmosphere)

In [ ]:
# Set up SMRT model configurations
ibam = make_model('iba', 'dort')

In [ ]:
# Set up sensor configuration
# Need to get as near to nadir as possible to tie in with airborne observations
rad = sensor_list.passive(89e9, 0)

In [ ]:
# Run model
base_res = ibam.run(rad, base_snowpack)

In [ ]:
tb_results(base_res.TbV(), single_frequency=89e9)

In [ ]:
# Get pit location info
import aesop
%autoreload
pits = aesop.get_pits()

In [ ]:
# Get flight data near pits
TB_all = aesop.get_marss_data(max_dist_in_km=0.25, theta=0, channel=0, pits=pits[pits.index=='A04N'])

In [ ]:
# Get ground-based TB
# Read in ground-based radiometer observations
# NB no date info in file
sbr = pd.read_csv('../Data/ground_obs/more89forMel.csv')

In [ ]:
# Get sbr data for all channels, all pits
tb89h0 = aesop.ground_based_data(sbr, '89H0', pits[pits.index=='A04N1'])
tb89v0 = aesop.ground_based_data(sbr, '89V0', pits[pits.index=='A04N1'])

### Experiment 1: modified debye relationship

In [ ]:
# pex = 0.8....1.2pc for depth hoar: Leinss et al, TC, 2019
mod_debye_snowpack = return_snowpack(A04N, with_ice_lens=True, dh_debye=1.2,
                                     flight_temperature_correction='C087',
                                     atmosphere=atmosphere)

In [ ]:
debye_res = ibam.run(rad, mod_debye_snowpack)

In [ ]:
tb_results(debye_res.TbV(), single_frequency=89e9)

### Experiment 2: Basal air gap

Put super low density layer of mostly air at base of snowpack

In [ ]:
airgap_snowpack = return_snowpack(A04N, with_ice_lens=True, airgap=True,
                                  flight_temperature_correction='C087',
                                  atmosphere=atmosphere)
airgap_res = ibam.run(rad, airgap_snowpack)

In [ ]:
tb_results(airgap_res.TbV(), single_frequency=89e9)

### Experiment 3: Max / Min density / SSA

In [ ]:
# Make list of snowpacks with
# 1. Low density, low SSA
# 2. Low density, high SSA
# 3. High density, low SSA
# 4. High density, high SSA
snowranges = [return_snowpack(A04N, with_ice_lens=True, density_extreme='low', 
                              ssa_extreme='low',atmosphere=atmosphere,
                              flight_temperature_correction='C087'),
             return_snowpack(A04N, with_ice_lens=True, density_extreme='low', 
                             ssa_extreme='high',atmosphere=atmosphere,
                             flight_temperature_correction='C087'),
             return_snowpack(A04N, with_ice_lens=True, density_extreme='high', 
                             ssa_extreme='low',atmosphere=atmosphere,
                             flight_temperature_correction='C087'),
             return_snowpack(A04N, with_ice_lens=True, density_extreme='high', 
                             ssa_extreme='high',atmosphere=atmosphere,
                             flight_temperature_correction='C087')]

In [ ]:
# Run model on list of snowpacks
range_res = ibam.run(rad, snowranges)

In [ ]:
# Low
tb_results(float(range_res.TbV()[0]), single_frequency=89e9)

In [ ]:
# High
tb_results(float(range_res.TbV()[3]), single_frequency=89e9)

### Experiment 4: Add surface crust

In [ ]:
surface_snowpack = return_snowpack(A04N, with_ice_lens=True, surface_crust=True,
                                   flight_temperature_correction='C087',
                                   atmosphere=atmosphere)
surface_res = ibam.run(rad, surface_snowpack)

In [ ]:
tb_results(surface_res.TbV(), single_frequency=89e9)

### Experiment 5: Variability in depth

Equal scaling of layer thicknesses

In [ ]:
# Make list of high depth / low depth snowpacks
depth_sensitive = [return_snowpack(A04N, with_ice_lens=True, 
                                   depth_uncertainty='low',
                                   atmosphere=atmosphere,
                                   flight_temperature_correction='C087'),
                   return_snowpack(A04N, with_ice_lens=True, 
                                   depth_uncertainty='high',
                                   atmosphere=atmosphere,
                                   flight_temperature_correction='C087')]

In [ ]:
depth_res = ibam.run(rad, depth_sensitive)

In [ ]:
# Low
tb_results(float(depth_res.TbV()[0]), single_frequency=89e9)

In [ ]:
# High
tb_results(float(depth_res.TbV()[1]), single_frequency=89e9)

### Plot results

In [ ]:
# Everything snowpack - lower limit
everything_low=return_snowpack(A04N, with_ice_lens=True, density_extreme='low', ssa_extreme='low', 
                surface_crust=True, airgap=True, dh_debye=1.2, depth_uncertainty='low',
                          flight_temperature_correction='C087',
                          atmosphere=atmosphere)

everything_low_res = ibam.run(rad, everything_low)

In [ ]:
tb_results(everything_low_res.TbV(), single_frequency=89e9)

In [ ]:
# Everything snowpack: upper limit
everything_high=return_snowpack(A04N, with_ice_lens=True, density_extreme='high', ssa_extreme='high', 
                surface_crust=True, airgap=True, dh_debye=1.2, depth_uncertainty='high',
                               flight_temperature_correction='C087',
                               atmosphere=atmosphere)

everything_high_res = ibam.run(rad, everything_high)

In [ ]:
tb_results(everything_high_res.TbV(), single_frequency=89e9)

Base case: DH and extremes

In [ ]:
# Everything snowpack: upper limit
base_low=return_snowpack(A04N, with_ice_lens=True, density_extreme='low', 
                         ssa_extreme='low', 
                         dh_debye=1.2,
                         flight_temperature_correction='C087',
                         atmosphere=atmosphere)
base_high=return_snowpack(A04N, with_ice_lens=True, density_extreme='high', 
                         ssa_extreme='high', 
                         dh_debye=1.2,
                         flight_temperature_correction='C087',
                         atmosphere=atmosphere)
base_low_res = ibam.run(rad, base_low)
base_high_res = ibam.run(rad, base_high)

In [ ]:
print (tb_results(base_low_res.TbV(), single_frequency=89e9))
print (tb_results(base_high_res.TbV(), single_frequency=89e9))

#### Get Observations

In [ ]:
obs_c087 = aesop.get_obs_per_aoi('C087')

In [ ]:
obs = obs_c087.loc['A04'[:3]]['M16-89'][obs_c087.loc['A04'[:3]]['M16-89'].topo_type=='plateau'].TB.values

In [ ]:
# Median
np.round(np.quantile(obs,0.5),2)

In [ ]:
# Min
np.round(np.min(obs),2)

In [ ]:
# Max
np.round(np.max(obs),2)

In [ ]:
# 25%ile
np.round(np.quantile(obs,0.25),2)

In [ ]:
# 75%ile
np.round(np.quantile(obs,0.75),2)